In [1]:
import pandas as pd


## Get data from Johns Hopkins

In [2]:
#!git clone https://github.com/CSSEGISandData/COVID-19.git 
!cd COVID-19 && git pull origin master

remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Total 47 (delta 23), reused 23 (delta 23), pack-reused 24
Unpacking objects: 100% (47/47), done.
From https://github.com/CSSEGISandData/COVID-19
 * branch            master     -> FETCH_HEAD
   66ca3a6..76b91e9  master     -> origin/master
Updating 66ca3a6..76b91e9
Fast-forward
 .../csse_covid_19_daily_reports/03-22-2020.csv     | 310 +++++++
 .../time_series_19-covid-Confirmed.csv             | 971 +++++++++++----------
 .../time_series_19-covid-Deaths.csv                | 971 +++++++++++----------
 .../time_series_19-covid-Recovered.csv             | 971 +++++++++++----------
 .../time_series_covid19_confirmed_global.csv       | 239 +++++
 .../time_series_covid19_deaths_global.csv          | 239 +++++
 6 files changed, 2252 insertions(+), 1449 deletions(-)
 create mode 100644 csse_covid_19_data/csse_covid_19_daily_reports/03-22-2020.csv
 create mode 100644 csse_covid_19_data/csse_covid_19_tim

In [3]:
df_corona_temp = pd.read_csv("./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df_corona_death_temp = pd.read_csv("./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")

### retreat data structure

In [4]:
def retreat_struct(df):
    output = df.groupby(by="Country/Region").sum()
    output.drop(['Lat','Long'], axis=1, inplace=True)
    output.fillna(0, inplace=True)
    output.index.name = None
    output.columns = pd.to_datetime(output.columns)
    output.rename(index={'Mainland China':'China', 'Korea, South': 'South Korea'},inplace=True)
    return output


In [5]:
df_corona_temp = retreat_struct(df_corona_temp)
df_corona_death_temp = retreat_struct(df_corona_death_temp)

In [6]:
df_corona_temp.shape, df_corona_death_temp.shape

((171, 61), (171, 61))

### Transform colums as lines

In [7]:
serie_corona = df_corona_temp.stack()
serie_death = df_corona_death_temp.stack()

In [8]:
# back to df 
df_corona = pd.DataFrame(serie_corona)

df_corona.reset_index(inplace=True)
df_corona.columns = ['country', 'date','case']

In [9]:
df_corona = df_corona.join(pd.DataFrame(serie_death),how='left',on=['country','date'])
df_corona.columns = ['country', 'date','case','death']

In [10]:
# Checking data 
df_corona.head()

,country,date,case,death
0,Afghanistan,2020-01-22,0,0
1,Afghanistan,2020-01-23,0,0
2,Afghanistan,2020-01-24,0,0
3,Afghanistan,2020-01-25,0,0
4,Afghanistan,2020-01-26,0,0


## UN Data on population and density

### Alignment of data

In [11]:
def align_country_names(df):
    ''' Align country names with CSSE reference'''
    output = df.rename(columns={'Region, subregion, country or area *' : 'Country'})
    output.set_index("Country", inplace=True)
    output.rename(index={'Republic of Korea' : 'South Korea', 'Iran (Islamic Republic of)': 'Iran', 'United States of America' : 'US', 'China, Taiwan Province of China' :'Taiwan','Russian Federation':'Russia','Viet Nam':'Vietnam'}, inplace=True)
    # focus only at country level data
    output = output[output['Type'] == 'Country/Area']
    # return 2020 data olny
    return output['2020']


### Population

In [12]:
df_wp_temp = pd.read_excel("./UN data/population_world.xlsx", sheet_name = 'ESTIMATES', skiprows= range(0,16))

In [13]:
serie_wp = align_country_names(df_wp_temp)
#check data
serie_wp

Country
Burundi                      11890.8
Comoros                      869.595
Djibouti                     988.002
Eritrea                      3546.43
Ethiopia                      114964
                              ...   
Bermuda                       62.273
Canada                       37742.2
Greenland                     56.772
Saint Pierre and Miquelon      5.795
US                            331003
Name: 2020, Length: 235, dtype: object

### Density data

In [14]:
serie_density = align_country_names(pd.read_excel("./UN data/WPP2019_POP_F06_POPULATION_DENSITY.xlsx", sheet_name = 'ESTIMATES', skiprows= range(0,16)))

In [15]:
# Check data
serie_density

Country
Burundi                       463.037
Comoros                       467.273
Djibouti                       42.623
Eritrea                       35.1131
Ethiopia                      114.964
                               ...   
Bermuda                       1245.46
Canada                        4.15045
Greenland                    0.138316
Saint Pierre and Miquelon     25.1957
US                            36.1854
Name: 2020, Length: 235, dtype: object

### Country infos
Crossing population and density and calculating size

In [16]:
df_1 = pd.DataFrame(serie_wp)
df_1.columns = ['population']
df_2 = pd.DataFrame(serie_density)
df_2.columns =['density']

In [17]:
# join two series as df
df_country_info = df_1.join(df_2)
# Calculate size of countries
df_country_info['size'] =  df_country_info['population']/df_country_info['density'] * 1000

In [18]:
# Cehcking data
df_country_info

,population,density,size
Country,,,
Burundi,11890.8,463.037,25680
Comoros,869.595,467.273,1861
Djibouti,988.002,42.623,23180
Eritrea,3546.43,35.1131,101000
Ethiopia,114964,114.964,1e+06
...,...,...,...
Bermuda,62.273,1245.46,50
Canada,37742.2,4.15045,9.09351e+06
Greenland,56.772,0.138316,410450


### Saving the UN data

In [19]:
df_country_info.to_csv('country_info.csv')

## Cross data CSSE and pop / density / size

In [20]:
df_dataset = df_corona.join(df_country_info, how='left',on='country')

In [21]:
df_dataset

,country,date,case,death,population,density,size
0,Afghanistan,2020-01-22,0,0,38928.3,59.6274,652860
1,Afghanistan,2020-01-23,0,0,38928.3,59.6274,652860
2,Afghanistan,2020-01-24,0,0,38928.3,59.6274,652860
3,Afghanistan,2020-01-25,0,0,38928.3,59.6274,652860
4,Afghanistan,2020-01-26,0,0,38928.3,59.6274,652860
...,...,...,...,...,...,...,...
10426,Zimbabwe,2020-03-18,0,0,14862.9,38.4204,386850
10427,Zimbabwe,2020-03-19,0,0,14862.9,38.4204,386850
10428,Zimbabwe,2020-03-20,1,0,14862.9,38.4204,386850
10429,Zimbabwe,2020-03-21,3,0,14862.9,38.4204,386850


In [22]:
## Calculate the age of crisis

In [23]:
ratio_crisis = 0.5
death_crisis = 10

In [24]:
df_dataset['ratio'] = df_dataset['case'] / df_dataset['population'] * 100
df_dataset['ratio_death'] = df_dataset['death'] / df_dataset['population'] * 100

In [25]:
# check data
df_dataset

,country,date,case,death,population,density,size,ratio,ratio_death
0,Afghanistan,2020-01-22,0,0,38928.3,59.6274,652860,0,0
1,Afghanistan,2020-01-23,0,0,38928.3,59.6274,652860,0,0
2,Afghanistan,2020-01-24,0,0,38928.3,59.6274,652860,0,0
3,Afghanistan,2020-01-25,0,0,38928.3,59.6274,652860,0,0
4,Afghanistan,2020-01-26,0,0,38928.3,59.6274,652860,0,0
...,...,...,...,...,...,...,...,...,...
10426,Zimbabwe,2020-03-18,0,0,14862.9,38.4204,386850,0,0
10427,Zimbabwe,2020-03-19,0,0,14862.9,38.4204,386850,0,0
10428,Zimbabwe,2020-03-20,1,0,14862.9,38.4204,386850,0.00672815,0
10429,Zimbabwe,2020-03-21,3,0,14862.9,38.4204,386850,0.0201844,0


### calculate ref data for crisis

In [26]:
serie_epidemic_start_date = df_dataset[df_dataset['ratio'] > ratio_crisis].groupby('country').min()['date']
serie_epidemic_death_start_date = df_dataset[df_dataset['death'] > death_crisis].groupby('country').min()['date']

In [27]:
serie_epidemic_death_start_date

country
Algeria          2020-03-20
Austria          2020-03-22
Belgium          2020-03-18
Brazil           2020-03-20
Canada           2020-03-20
China            2020-01-22
Denmark          2020-03-21
Ecuador          2020-03-22
Egypt            2020-03-22
France           2020-03-07
Germany          2020-03-15
Greece           2020-03-21
Indonesia        2020-03-18
Iran             2020-02-24
Iraq             2020-03-17
Italy            2020-02-26
Japan            2020-03-11
Netherlands      2020-03-14
Philippines      2020-03-15
Portugal         2020-03-21
San Marino       2020-03-18
South Korea      2020-02-26
Spain            2020-03-08
Sweden           2020-03-19
Switzerland      2020-03-13
Turkey           2020-03-22
US               2020-03-04
United Kingdom   2020-03-14
Name: date, dtype: datetime64[ns]

In [28]:
# cases
df_epidemic_start = pd.DataFrame(serie_epidemic_start_date)
df_epidemic_start.columns = ['start_date']
# deaths
df_epidemic_death_start = pd.DataFrame(serie_epidemic_death_start_date)
df_epidemic_death_start.columns = ['start_death_date']

### add start date to dataset

In [29]:
df_dataset = df_dataset.join(df_epidemic_start, how='right', on='country')
df_dataset = df_dataset.join(df_epidemic_death_start, how='right', on='country')

In [30]:
df_dataset['age'] = (df_dataset['date'] - df_dataset['start_date']).dt.days
df_dataset['age_death'] = (df_dataset['date'] - df_dataset['start_death_date']).dt.days

In [31]:
#check data
df_dataset

,country,date,case,death,population,density,size,ratio,ratio_death,start_date,start_death_date,age,age_death
549.0,Austria,2020-01-22,0.0,0.0,9006.4,109.289,82409,0,0,2020-03-06,2020-03-22,-44.0,-60.0
550.0,Austria,2020-01-23,0.0,0.0,9006.4,109.289,82409,0,0,2020-03-06,2020-03-22,-43.0,-59.0
551.0,Austria,2020-01-24,0.0,0.0,9006.4,109.289,82409,0,0,2020-03-06,2020-03-22,-42.0,-58.0
552.0,Austria,2020-01-25,0.0,0.0,9006.4,109.289,82409,0,0,2020-03-06,2020-03-22,-41.0,-57.0
553.0,Austria,2020-01-26,0.0,0.0,9006.4,109.289,82409,0,0,2020-03-06,2020-03-22,-40.0,-56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10064.0,United Kingdom,2020-03-22,5741.0,282.0,67886,280.602,241930,8.45682,0.415402,2020-03-10,2020-03-14,12.0,8.0
NaN,Algeria,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2020-03-20,NaN,NaN
NaN,Egypt,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2020-03-22,NaN,NaN
NaN,Indonesia,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2020-03-18,NaN,NaN


## Saving final data

In [32]:
df_dataset.to_csv('dataset.csv')